In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(1)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x)  

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 8

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
# Deep Ritz Method
def DRM(x):
    u_hat = model(x)
    ux = torch.zeros(x.size()[0], dimension).cuda()
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        ux[:, i] = (model(x+step_size*dx) - model(x-step_size*dx))[:,0]/step_size/2
    uxsq = (torch.sum(ux**2, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum(0.5 * uxsq + 0.5*pi**2*u_hat**2 - f_temp*u_hat)/x.size()[0]
    
    Nb = 1000
    xa1 = 2*torch.rand(Nb, dimension) - 1.0
    xa1[:, 0] = -torch.ones(Nb)
    
    xa2 = 2*torch.rand(Nb, dimension) - 1.0
    xa2[:, 0:8] = xa1[:, 0:8]
    xa2[:, 0] = torch.ones(Nb)
    
    xb1 = 2*torch.rand(Nb, dimension) - 1.0
    xb1[:, 1] = -torch.ones(Nb)
    
    xb2 = 2*torch.rand(Nb, dimension) - 1.0
    xb2[:, 0:8]  = xb1[:, 0:8] 
    xb2[:, 1] = torch.ones(Nb)
    
    xc1 = 2*torch.rand(Nb, dimension) - 1.0
    xc1[:, 2] = -torch.ones(Nb)
    
    xc2 = 2*torch.rand(Nb, dimension) - 1.0
    xc2[:, 0:8] = xc1[:, 0:8]
    xc2[:, 2] = torch.ones(Nb)
    
    xd1 = 2*torch.rand(Nb, dimension) - 1.0
    xd1[:, 3] = -torch.ones(Nb)
    
    xd2 = 2*torch.rand(Nb, dimension) - 1.0
    xd2[:, 0:8] = xd1[:, 0:8]
    xd2[:, 3] = torch.ones(Nb)
    
    xe1 = 2*torch.rand(Nb, dimension) - 1.0
    xe1[:, 4] = -torch.ones(Nb)
    
    xe2 = 2*torch.rand(Nb, dimension) - 1.0
    xe2[:, 0:8] = xe1[:, 0:8]
    xe2[:, 4] = torch.ones(Nb)
    
    xf1 = 2*torch.rand(Nb, dimension) - 1.0
    xf1[:, 5] = -torch.ones(Nb)
    
    xf2 = 2*torch.rand(Nb, dimension) - 1.0
    xf2[:, 0:8] = xf1[:, 0:8] 
    xf2[:, 5] = torch.ones(Nb)
    
    xg1 = 2*torch.rand(Nb, dimension) - 1.0
    xg1[:, 6] = -torch.ones(Nb)
    
    xg2 = 2*torch.rand(Nb, dimension) - 1.0
    xg2[:, 0:8] = xg1[:, 0:8] 
    xg2[:, 6] = torch.ones(Nb)
    
    xh1 = 2*torch.rand(Nb, dimension) - 1.0
    xh1[:, 7] = -torch.ones(Nb)
    
    xh2 = 2*torch.rand(Nb, dimension) - 1.0
    xh2[:, 0:8] = xh1[:, 0:8]
    xh2[:, 7] = torch.ones(Nb)
    
    xa1 = xa1.cuda()
    xa2 = xa2.cuda()
    xb1 = xb1.cuda()
    xb2 = xb2.cuda()
    xc1 = xc1.cuda()
    xc2 = xc2.cuda()
    xd1 = xd1.cuda()
    xd2 = xd2.cuda()
    xe1 = xe1.cuda()
    xe2 = xe2.cuda()
    xf1 = xf1.cuda()
    xf2 = xf2.cuda()
    xg1 = xg1.cuda()
    xg2 = xg2.cuda()
    xh1 = xh1.cuda()
    xh2 = xh2.cuda()
    
    pxa = ((model(xa1) - model(xa2))**2).cuda() 
    pxb = ((model(xb1) - model(xb2))**2).cuda()  
    pxc = ((model(xc1) - model(xc2))**2).cuda()  
    pxd = ((model(xd1) - model(xd2))**2).cuda()  
    pxe = ((model(xe1) - model(xe2))**2).cuda() 
    pxf = ((model(xf1) - model(xf2))**2).cuda()  
    pxg = ((model(xg1) - model(xg2))**2).cuda()  
    pxh = ((model(xh1) - model(xh2))**2).cuda()
    
    part_2 = torch.sum((pxa + pxb + pxc + pxd + pxe + pxf + pxg + pxh)[:,0])/2/xa1.size()[0]/8
    
    dx1 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx1[:, 0] = torch.ones(xb1.size()[0])
    dx2 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx2[:, 1] = torch.ones(xb1.size()[0])
    dx3 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx3[:, 2] = torch.ones(xb1.size()[0])
    dx4 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx4[:, 3] = torch.ones(xb1.size()[0])
    dx5 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx5[:, 4] = torch.ones(xb1.size()[0])
    dx6 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx6[:, 5] = torch.ones(xb1.size()[0])
    dx7 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx7[:, 6] = torch.ones(xb1.size()[0])
    dx8 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx8[:, 7] = torch.ones(xb1.size()[0])
    
    dxa = (((model(xa1+step_size*dx1) - model(xa1-step_size*dx1))/step_size/2 - (model(xa2+step_size*dx1) - model(xa2-step_size*dx1))/step_size/2)**2).cuda()  
    dxb = (((model(xb1+step_size*dx2) - model(xb1-step_size*dx2))/step_size/2 - (model(xb2+step_size*dx2) - model(xb2-step_size*dx2))/step_size/2)**2).cuda()  
    dxc = (((model(xc1+step_size*dx3) - model(xc1-step_size*dx3))/step_size/2 - (model(xc2+step_size*dx3) - model(xc2-step_size*dx3))/step_size/2)**2).cuda()  
    dxd = (((model(xd1+step_size*dx4) - model(xd1-step_size*dx4))/step_size/2 - (model(xd2+step_size*dx4) - model(xd2-step_size*dx4))/step_size/2)**2).cuda()  
    dxe = (((model(xe1+step_size*dx5) - model(xe1-step_size*dx5))/step_size/2 - (model(xe2+step_size*dx5) - model(xe2-step_size*dx5))/step_size/2)**2).cuda()  
    dxf = (((model(xf1+step_size*dx6) - model(xf1-step_size*dx6))/step_size/2 - (model(xf2+step_size*dx6) - model(xf2-step_size*dx6))/step_size/2)**2).cuda()  
    dxg = (((model(xg1+step_size*dx7) - model(xg1-step_size*dx7))/step_size/2 - (model(xg2+step_size*dx7) - model(xg2-step_size*dx7))/step_size/2)**2).cuda()  
    dxh = (((model(xh1+step_size*dx8) - model(xh1-step_size*dx8))/step_size/2 - (model(xh2+step_size*dx8) - model(xh2-step_size*dx8))/step_size/2)**2).cuda()  
    
    
    part_3 = torch.sum((dxa + dxb + dxc + dxd + dxe + dxf + dxg + dxh)[:,0])/2/xb1.size()[0]/8
    
    lambda1 = 1.0
    lambda2 = 0.5
    return part_1 + lambda1 * part_2 + lambda2 * part_3
    

In [10]:
Data_size = 4000
def Gendata():
    x = 2*torch.rand(Data_size, dimension) - 1.0
    return x.cuda()

In [11]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = DeepRitzNet(dimension, 16)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:1" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=8, out_features=16, bias=True)
  (linear1): Linear(in_features=16, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=16, bias=True)
  (linear3): Linear(in_features=16, out_features=16, bias=True)
  (linear4): Linear(in_features=16, out_features=16, bias=True)
  (linear5): Linear(in_features=16, out_features=16, bias=True)
  (linear6): Linear(in_features=16, out_features=16, bias=True)
  (linear_out): Linear(in_features=16, out_features=1, bias=True)
)

In [14]:
import torch.optim as optim
import torch.nn as nn
import time

In [15]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact)**2))/torch.sqrt(torch.sum((exact)**2))
    return value

In [16]:
traintime = 50000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [17]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_8D_Periodic.npy", error_save)
np.save("DRM_relative_error_8D_Periodic.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(20514.9838, device='cuda:1')
current relative error is:  tensor(26.5891, device='cuda:1')
current epoch is:  50
current loss is:  tensor(825.5890, device='cuda:1')
current relative error is:  tensor(4.7678, device='cuda:1')
current epoch is:  100
current loss is:  tensor(237.8717, device='cuda:1')
current relative error is:  tensor(2.4027, device='cuda:1')
current epoch is:  150
current loss is:  tensor(124.2156, device='cuda:1')
current relative error is:  tensor(1.7028, device='cuda:1')
current epoch is:  200
current loss is:  tensor(84.2844, device='cuda:1')
current relative error is:  tensor(1.5562, device='cuda:1')
current epoch is:  250
current loss is:  tensor(65.2247, device='cuda:1')
current relative error is:  tensor(1.3712, device='cuda:1')
current epoch is:  300
current loss is:  tensor(47.3627, device='cuda:1')
current relative error is:  tensor(1.2614, device='cuda:1')
current epoch is:  350
current loss is:  tensor(40.8947, d

current epoch is:  3050
current loss is:  tensor(-18.9912, device='cuda:1')
current relative error is:  tensor(0.5864, device='cuda:1')
current epoch is:  3100
current loss is:  tensor(-20.0295, device='cuda:1')
current relative error is:  tensor(0.5816, device='cuda:1')
current epoch is:  3150
current loss is:  tensor(-20.5141, device='cuda:1')
current relative error is:  tensor(0.5821, device='cuda:1')
current epoch is:  3200
current loss is:  tensor(-20.2153, device='cuda:1')
current relative error is:  tensor(0.5687, device='cuda:1')
current epoch is:  3250
current loss is:  tensor(-19.4431, device='cuda:1')
current relative error is:  tensor(0.5583, device='cuda:1')
current epoch is:  3300
current loss is:  tensor(-21.4274, device='cuda:1')
current relative error is:  tensor(0.5500, device='cuda:1')
current epoch is:  3350
current loss is:  tensor(-21.5428, device='cuda:1')
current relative error is:  tensor(0.5508, device='cuda:1')
current epoch is:  3400
current loss is:  tensor

current epoch is:  6100
current loss is:  tensor(-33.4901, device='cuda:1')
current relative error is:  tensor(0.2883, device='cuda:1')
current epoch is:  6150
current loss is:  tensor(-32.0857, device='cuda:1')
current relative error is:  tensor(0.2861, device='cuda:1')
current epoch is:  6200
current loss is:  tensor(-34.4703, device='cuda:1')
current relative error is:  tensor(0.2841, device='cuda:1')
current epoch is:  6250
current loss is:  tensor(-33.3407, device='cuda:1')
current relative error is:  tensor(0.2812, device='cuda:1')
current epoch is:  6300
current loss is:  tensor(-31.4067, device='cuda:1')
current relative error is:  tensor(0.2675, device='cuda:1')
current epoch is:  6350
current loss is:  tensor(-33.8403, device='cuda:1')
current relative error is:  tensor(0.2662, device='cuda:1')
current epoch is:  6400
current loss is:  tensor(-31.4180, device='cuda:1')
current relative error is:  tensor(0.2652, device='cuda:1')
current epoch is:  6450
current loss is:  tensor

current epoch is:  9150
current loss is:  tensor(-37.0543, device='cuda:1')
current relative error is:  tensor(0.1266, device='cuda:1')
current epoch is:  9200
current loss is:  tensor(-35.5386, device='cuda:1')
current relative error is:  tensor(0.1305, device='cuda:1')
current epoch is:  9250
current loss is:  tensor(-36.1197, device='cuda:1')
current relative error is:  tensor(0.1290, device='cuda:1')
current epoch is:  9300
current loss is:  tensor(-34.0789, device='cuda:1')
current relative error is:  tensor(0.1275, device='cuda:1')
current epoch is:  9350
current loss is:  tensor(-36.8365, device='cuda:1')
current relative error is:  tensor(0.1227, device='cuda:1')
current epoch is:  9400
current loss is:  tensor(-39.9047, device='cuda:1')
current relative error is:  tensor(0.1290, device='cuda:1')
current epoch is:  9450
current loss is:  tensor(-39.4600, device='cuda:1')
current relative error is:  tensor(0.1735, device='cuda:1')
current epoch is:  9500
current loss is:  tensor

current epoch is:  12150
current loss is:  tensor(-38.0040, device='cuda:1')
current relative error is:  tensor(0.0963, device='cuda:1')
current epoch is:  12200
current loss is:  tensor(-38.7509, device='cuda:1')
current relative error is:  tensor(0.1105, device='cuda:1')
current epoch is:  12250
current loss is:  tensor(-36.2034, device='cuda:1')
current relative error is:  tensor(0.1219, device='cuda:1')
current epoch is:  12300
current loss is:  tensor(-36.8024, device='cuda:1')
current relative error is:  tensor(0.0988, device='cuda:1')
current epoch is:  12350
current loss is:  tensor(-38.6949, device='cuda:1')
current relative error is:  tensor(0.1332, device='cuda:1')
current epoch is:  12400
current loss is:  tensor(-38.0355, device='cuda:1')
current relative error is:  tensor(0.1087, device='cuda:1')
current epoch is:  12450
current loss is:  tensor(-36.5787, device='cuda:1')
current relative error is:  tensor(0.1241, device='cuda:1')
current epoch is:  12500
current loss is:

...

current epoch is:  36150
current loss is:  tensor(-37.6076, device='cuda:1')
current relative error is:  tensor(0.0642, device='cuda:1')
current epoch is:  36200
current loss is:  tensor(-40.4101, device='cuda:1')
current relative error is:  tensor(0.0696, device='cuda:1')
current epoch is:  36250
current loss is:  tensor(-38.6170, device='cuda:1')
current relative error is:  tensor(0.0564, device='cuda:1')
current epoch is:  36300
current loss is:  tensor(-39.0234, device='cuda:1')
current relative error is:  tensor(0.0712, device='cuda:1')
current epoch is:  36350
current loss is:  tensor(-38.4899, device='cuda:1')
current relative error is:  tensor(0.0802, device='cuda:1')
current epoch is:  36400
current loss is:  tensor(-37.6881, device='cuda:1')
current relative error is:  tensor(0.0779, device='cuda:1')
current epoch is:  36450
current loss is:  tensor(-38.0726, device='cuda:1')
current relative error is:  tensor(0.0607, device='cuda:1')
current epoch is:  36500
current loss is:

current epoch is:  39150
current loss is:  tensor(-39.9266, device='cuda:1')
current relative error is:  tensor(0.0548, device='cuda:1')
current epoch is:  39200
current loss is:  tensor(-38.2516, device='cuda:1')
current relative error is:  tensor(0.0526, device='cuda:1')
current epoch is:  39250
current loss is:  tensor(-37.4782, device='cuda:1')
current relative error is:  tensor(0.0678, device='cuda:1')
current epoch is:  39300
current loss is:  tensor(-39.9602, device='cuda:1')
current relative error is:  tensor(0.0693, device='cuda:1')
current epoch is:  39350
current loss is:  tensor(-36.9724, device='cuda:1')
current relative error is:  tensor(0.0691, device='cuda:1')
current epoch is:  39400
current loss is:  tensor(-38.8547, device='cuda:1')
current relative error is:  tensor(0.0600, device='cuda:1')
current epoch is:  39450
current loss is:  tensor(-39.3019, device='cuda:1')
current relative error is:  tensor(0.0687, device='cuda:1')
current epoch is:  39500
current loss is:

current epoch is:  42150
current loss is:  tensor(-37.8202, device='cuda:1')
current relative error is:  tensor(0.0627, device='cuda:1')
current epoch is:  42200
current loss is:  tensor(-38.0139, device='cuda:1')
current relative error is:  tensor(0.0903, device='cuda:1')
current epoch is:  42250
current loss is:  tensor(-39.3160, device='cuda:1')
current relative error is:  tensor(0.0808, device='cuda:1')
current epoch is:  42300
current loss is:  tensor(-40.3973, device='cuda:1')
current relative error is:  tensor(0.0845, device='cuda:1')
current epoch is:  42350
current loss is:  tensor(-39.1232, device='cuda:1')
current relative error is:  tensor(0.0686, device='cuda:1')
current epoch is:  42400
current loss is:  tensor(-41.4993, device='cuda:1')
current relative error is:  tensor(0.0707, device='cuda:1')
current epoch is:  42450
current loss is:  tensor(-36.8403, device='cuda:1')
current relative error is:  tensor(0.0842, device='cuda:1')
current epoch is:  42500
current loss is:

current epoch is:  45150
current loss is:  tensor(-38.1030, device='cuda:1')
current relative error is:  tensor(0.0525, device='cuda:1')
current epoch is:  45200
current loss is:  tensor(-37.6944, device='cuda:1')
current relative error is:  tensor(0.0735, device='cuda:1')
current epoch is:  45250
current loss is:  tensor(-41.7322, device='cuda:1')
current relative error is:  tensor(0.0682, device='cuda:1')
current epoch is:  45300
current loss is:  tensor(-39.5874, device='cuda:1')
current relative error is:  tensor(0.0562, device='cuda:1')
current epoch is:  45350
current loss is:  tensor(-40.7308, device='cuda:1')
current relative error is:  tensor(0.0739, device='cuda:1')
current epoch is:  45400
current loss is:  tensor(-35.5518, device='cuda:1')
current relative error is:  tensor(0.1051, device='cuda:1')
current epoch is:  45450
current loss is:  tensor(-39.4772, device='cuda:1')
current relative error is:  tensor(0.0902, device='cuda:1')
current epoch is:  45500
current loss is:

current epoch is:  48150
current loss is:  tensor(-42.2054, device='cuda:1')
current relative error is:  tensor(0.0663, device='cuda:1')
current epoch is:  48200
current loss is:  tensor(-39.1579, device='cuda:1')
current relative error is:  tensor(0.0733, device='cuda:1')
current epoch is:  48250
current loss is:  tensor(-38.5025, device='cuda:1')
current relative error is:  tensor(0.0681, device='cuda:1')
current epoch is:  48300
current loss is:  tensor(-40.9018, device='cuda:1')
current relative error is:  tensor(0.0752, device='cuda:1')
current epoch is:  48350
current loss is:  tensor(-39.8918, device='cuda:1')
current relative error is:  tensor(0.0641, device='cuda:1')
current epoch is:  48400
current loss is:  tensor(-37.4201, device='cuda:1')
current relative error is:  tensor(0.0481, device='cuda:1')
current epoch is:  48450
current loss is:  tensor(-38.2055, device='cuda:1')
current relative error is:  tensor(0.0608, device='cuda:1')
current epoch is:  48500
current loss is: